<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/tmp_for_martinelli/raster_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports and modules.
%pip install opencv-python
%pip install matplotlib
%pip install pandas

from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import matplotlib.animation as animation
from matplotlib import rc
from typing import List
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
from tqdm import tqdm
from io import StringIO
import xgboost as xgb
import os
import math
import glob
import tensorflow as tf
import joblib

rc('animation', html='jshtml')

import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_source_control_pane()
# ddfimport.ddf_import_common()

In [ ]:
import raster
import importlib
import pytest

importlib.reload(raster)

def test_coords_to_indices():
  bounds = raster.Bounds(50, 100, 50, 100, 1, 1, 50, 50)
  x, y = raster.coords_to_indices(bounds, 55, 55)
  assert x == 45
  assert y == 5

  bounds = raster.Bounds(-100, -50, -100, -50, 1, 1, 50, 50)
  x, y = raster.coords_to_indices(bounds, -55, -55)
  assert x == 5
  assert y == 45

  bounds = raster.Bounds(-10, 50, -10, 50, 1, 1, 60, 60)
  x, y = raster.coords_to_indices(bounds, -1, 13)
  assert x == 37
  assert y == 9

  bounds = raster.Bounds(minx=-73.97513931345594, maxx=-34.808472803053895, miny=-33.73347244751509, maxy=5.266527396029211, pixel_size_x=0.04166666650042771, pixel_size_y=-0.041666666499513144, raster_size_x=937, raster_size_y=941)
  x, y = raster.coords_to_indices(bounds, -67.14342073173958, -7.273271869467912e-05)
  #print(x)
  assert x == 131 # was: 132
  assert y == 163

def test_mount_gdrive_local():
  raster.GDRIVE_BASE = "/content/gdrive"
  raster.mount_gdrive()
  assert os.path.exists(raster.GDRIVE_BASE)

def test_is_valid_point():
  assert(not raster.is_valid_point(0, 0,
                                   raster.atmosphere_isoscape_geotiff()))
  assert(raster.is_valid_point(-2.5, -59,
                                   raster.atmosphere_isoscape_geotiff()))

def smoke_test_gen_isoscape():
  model = tf.keras.saving.load_model("/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/unit_tests/test_model.tf")
  transformer =  joblib.load("/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/unit_tests/test_model.pkl")
  required_geotiffs = ["Mean Annual Precipitation"]
  run_id = 'raster_unit_test'
  raster.generate_isoscapes_from_variational_model(
      run_id, model, transformer, required_geotiffs, True)

  generated_means_raster = raster.load_raster(raster.get_raster_path(run_id+".tiff"), use_only_band_index=0)
  assert 25.29 == pytest.approx(generated_means_raster.value_at(-59, -2.5), 0.01)
  assert(not raster.is_valid_point(0, 0, generated_means_raster))

  generated_vars_raster = raster.load_raster(raster.get_raster_path(run_id+".tiff"), use_only_band_index=1)
  assert 1.54 == pytest.approx(generated_vars_raster.value_at(-59, -2.5), 0.01)
  assert(not raster.is_valid_point(0, 0, generated_vars_raster))

  anim = raster.animate(raster.load_raster(raster.get_raster_path(run_id+".tiff"), use_only_band_index=0), 1, 1)
  anim

test_coords_to_indices()
test_is_valid_point()
test_mount_gdrive_local()
smoke_test_gen_isoscape()
